In [12]:
from bokeh.io import output_file, show, output_notebook, save
from bokeh.plotting import figure

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.widgets import Tabs, Panel 
from bokeh.transform import factor_cmap
from bokeh.palettes import *
from bokeh.themes import built_in_themes
from bokeh.io import curdoc


import pandas as pd
import numpy as np 

import geopandas as gpd
from pathlib import Path
import matplotlib.colors

In [13]:
output_notebook()

Loading BokehJS ...

In [14]:
micropolis_folder = Path('../../infrarisk/data/networks/micropolis/gis')

In [15]:
p_power = figure(background_fill_color="white", 
           plot_width=1500, 
           height = 1200,
           title = 'Micropolis Power Network',
           x_range=(1000, 8000), 
           y_range=(1000, 6600))

p_water = figure(background_fill_color="white", 
           plot_width=1500, 
           height = 1200,
           title = 'Micropolis Water Network',
           x_range=(1000, 8000), 
           y_range=(1000, 6600))

p_transpo = figure(background_fill_color="white", 
           plot_width=1500, 
           height = 1200,
           title = 'Micropolis Transportation Network',
           x_range=(1000, 8000), 
           y_range=(1000, 6600))

In [16]:
power_color = 'dimgray'
line_width = 1.5

transpo_color = 'dimgray'
transpo_line_width = 1.5

water_color = 'red'
water_width = 1.5

palette = Plasma[6][0:4] #YlGnBu[9][0:4] #YlGnBu

buildings_alpha = 1
building_pal = Set3

In [17]:
building_pal[5]

('#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3')

In [18]:
Category20[3][0]
#RdYlGn[4]

'#1f77b4'

# Required Shapefiles

## Buildings

In [19]:
#micropolis buildings
micropolis_buildings = gpd.read_file(micropolis_folder/'buildings_v1.shp', encoding="utf-8")

## Power

In [20]:
#Micropolis buses
micropolis_buses = gpd.read_file(micropolis_folder/'all_buses.shp', encoding="utf-8")
micropolis_buses['x'] = micropolis_buses.geometry.x
micropolis_buses['y'] = micropolis_buses.geometry.y

#micropolis_substation
micropolis_substation = gpd.read_file(micropolis_folder/'micropolis_substation.shp', encoding="utf-8")

#Lines
micropolis_links = gpd.read_file(micropolis_folder/'combined_lines.shp', encoding="utf-8")
micropolis_links.id = micropolis_links.id.values.astype(int)
micropolis_links['from'] = micropolis_links['from'].values.astype(int)
micropolis_links['to'] = micropolis_links['to'].values.astype(int)

micropolis_lines = micropolis_links[micropolis_links.layer.isin(['Feeder1_lines', 'Feeder2_lines', 'Overhead lines', 'Underground lines'])]

#switches
micropolis_switches = micropolis_links[micropolis_links.layer == 'Switches']
micropolis_subtransformer = micropolis_links[micropolis_links.layer == 'Transformers']

#Off switches
micropolis_offswitches = micropolis_links[micropolis_links.layer == 'Off Switch']

#transformers

micropolis_hub_dist = gpd.read_file(micropolis_folder/'Hub distance.shp', encoding="utf-8")
micropolis_hub_dist1 = micropolis_hub_dist.loc[:, ['ID', 'HubName', 'HubDist']]
micropolis_buildings= pd.merge(micropolis_buildings, micropolis_hub_dist1, left_on = 'ID', right_on = 'ID', how = 'inner')

micropolis_power_details = pd.read_csv(micropolis_folder/'building_power_details.csv')
micropolis_buildings= pd.merge(micropolis_buildings, micropolis_power_details, left_on = 'FACILITY_T', right_on = 'building', how = 'inner')
del micropolis_buildings['building']
micropolis_buildings.HubName = micropolis_buildings.HubName.astype(int)

In [21]:
micropolis_buses.layer.unique()

array(['Feeder1 buses', 'Overhead distribution buses', 'Feeder2_buses',
       'Underground distribution buses', 'Substation buses'], dtype=object)

## Transportation

In [22]:
#function to find nearest coordinate
def find_nearest_value(number, setted_list):
    minimum = float("inf")
    for val in setted_list:
        if abs(val - number) < minimum:
            final_value = val
            minimum = abs(val - number)
    return final_value, number

x =  [687, 1186, 2392, 2796, 3203, 2604, 4806, 5602, 7208, 7600, 8442, 6107, 5386]
find_nearest_value(3000, x)

(3203, 3000)

In [23]:
x = [687, 1186, 2392, 2796, 3203, 3604, 4806, 5602, 7208, 7600, 8442, 6107, 5386]
y = [1207, 2392, 2806, 3218, 3603, 3983, 4412, 4787, 5050, 5323, 5640, 6013, 6405]

# transportation nodes
nodes = gpd.read_file(micropolis_folder/'transpo_junctions.shp', encoding="utf-8")
nodes.id = 'T_J' + nodes.id.astype(str)
#[print(find_nearest_value(x_coord, x)) for x_coord in nodes.centroid.x]
nodes['x'] = [find_nearest_value(x_coord, x)[0] for x_coord in nodes.centroid.x]
nodes['y'] = [find_nearest_value(y_coord, y)[0] for y_coord in nodes.centroid.y]

#links
links = gpd.read_file(micropolis_folder/'transpo_links.shp', encoding="utf-8")
links.id = 'T_L' + links['id'].astype(str)
links['from'] = 'T_J' + links['from'].astype(str)
links.to = 'T_J' + links.to.astype(str)
del links['geometry']

C:\Users\srijith\AppData\Local\Temp/ipykernel_4536/3240905435.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nodes['x'] = [find_nearest_value(x_coord, x)[0] for x_coord in nodes.centroid.x]
C:\Users\srijith\AppData\Local\Temp/ipykernel_4536/3240905435.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nodes['y'] = [find_nearest_value(y_coord, y)[0] for y_coord in nodes.centroid.y]


## Water

In [24]:
# water nodes
micropolis_waternodes = pd.read_csv(micropolis_folder/'micropolis_waternodes.csv')
#micropolis_waternodes = micropolis_waternodes[micropolis_waternodes.category != 'Pump']


#water pipes
micropolis_waterlinks = pd.read_csv(micropolis_folder/'micropolis_waterpipes.csv')
micropolis_pipes = micropolis_waterlinks[micropolis_waterlinks.category != "Pump"]

micropolis_pumps = micropolis_waterlinks[micropolis_waterlinks.category == "Pump"]

#water streams
micropolis_streams = gpd.read_file(micropolis_folder/'streams_v1.shp', encoding="utf-8")
micropolis_streams.ID = micropolis_streams.index + 1

#water bodies
micropolis_waterbodies = gpd.read_file(micropolis_folder/'waterbody_v1.shp', encoding="utf-8")

# Micropolis Transportation Network

In [25]:
# junctions
x, y, junc_layer, id = [], [], [], []
[(x.append(row['x']), 
  y.append(row['y']),
  junc_layer.append('Transportation junctions'), 
  id.append(row['id'])) for index, row in nodes.iterrows()]

plot_transpo_juncs = p_transpo.circle('x', 'y', source = ColumnDataSource(dict(x = x, y = y, junc_layer = junc_layer, id = id)), 
                    color=transpo_color, alpha = 0.6,
                    muted_color=transpo_color, muted_alpha=0.2,
                    size = 5, 
                    legend_field="junc_layer")

In [26]:
# Transportation links
x, y, link_layer, link_category, id = [], [], [], [], []
[
  (x.append(
    [nodes[nodes['id'] == row['from']].x, 
     nodes[nodes['id'] == row['to']].x
     ]
    ), 
  y.append(
    [nodes[nodes['id'] == row['from']].y,
     nodes[nodes['id'] == row['to']].y
    ]
    ),
  link_layer.append(
    'Transportation links'
    ),
  link_category.append(
    row['Category']
    ),
  id.append(
    row['id']
    )
  ) for index, row in links.iterrows()]

road_palette = factor_cmap('link_category', palette = palette[0:3], factors = np.unique(np.array(link_category)))

plot_transpo_links = p_transpo.multi_line('x', 'y', source = ColumnDataSource(dict(x = x, y = y, 
      link_layer = link_layer, link_category = link_category, id = id)), color = road_palette, line_alpha = 1, line_width = transpo_line_width,
                                muted_color=road_palette, muted_alpha=0.2,
                                line_color = road_palette,
                                legend_field="link_category")

In [27]:
#buildings
x, y, facility, building_type, population, load, connection, building_id = [], [], [], [], [], [], [], []
[
  (x.append(
    list(row['geometry'].exterior.coords.xy[0])), 
  y.append(
    list(row['geometry'].exterior.coords.xy[1])),
  facility.append(
    row['FACILITY_T']),
  building_type.append(
    row['type']),
  population.append(
    row['POP']),
  load.append(
    row['Pabc']),
  connection.append(
    row['phases']),
  building_id.append(
    row['ID'])
  ) for index, row in micropolis_buildings.iterrows() ]

building_palette = factor_cmap('type', palette = building_pal[5], factors = np.unique(np.array(building_type)))

plot_buildings = p_transpo.patches('x', 'y', source = ColumnDataSource(dict(x = x, y = y, 
                 facility = facility, type = building_type, population = population, 
                 load = load, connection = connection, id = building_id)), 
                           legend_field="type",
                           fill_color= building_palette,
                           fill_alpha = buildings_alpha,  
                           line_color = "white", 
                           muted_color=building_palette, muted_alpha=0.2,
                           line_width = 0.7)

In [28]:
building_hover = HoverTool(renderers=[plot_buildings])
building_hover.tooltips = [('Building ID', 'P_LO@id'), ('Building type', '@facility'), ('Zone', '@type'), ('Population', '@population'),
                           ('Load', '@load kW'), ('Connection type', '@connection-phase')]
p_transpo.add_tools(building_hover)


transpo_junc_hover = HoverTool(renderers=[plot_transpo_juncs])
transpo_junc_hover.tooltips = [('Node ID', '@id'), ('Category', '@junc_layer')]
p_transpo.add_tools(transpo_junc_hover)

transpo_links_hover = HoverTool(renderers=[plot_transpo_links])
transpo_links_hover.tooltips = [('Link ID', '@id'), ('Class', '@link_category'), ('Layer', '@link_layer'), ('Type', 'Bi-directional')]
p_transpo.add_tools(transpo_links_hover)

In [29]:
p_transpo.legend.location = "top_left"
p_transpo.legend.click_policy="mute"
p_transpo.legend.background_fill_color = "gainsboro"
p_transpo.legend.background_fill_alpha = 0.2

In [30]:
micropolis_buses.layer.unique()

array(['Feeder1 buses', 'Overhead distribution buses', 'Feeder2_buses',
       'Underground distribution buses', 'Substation buses'], dtype=object)

# Micropolis Power Network

In [31]:
# buses
micropolis_normalbuses = micropolis_buses[micropolis_buses.layer != 'Substation buses']
x, y, bus_layer, id = [], [], [], []
[(x.append(row['x']), 
  y.append(row['y']),
  bus_layer.append('Buses'), 
  id.append(row['FID'])) for index, row in micropolis_normalbuses.iterrows()]

plot_bus = p_power.square('x', 'y', source = ColumnDataSource(dict(x = x, y = y, layer = bus_layer, id = id)), 
                    color=power_color, 
                    muted_color=power_color, muted_alpha=0.2,
                    size = 5, 
                    legend_field="layer")

In [32]:
#substation
x, y, substation_layer, substation_id = [], [], [], []
[
  (x.append(
    list(row['geometry'].exterior.coords.xy[0])), 
  y.append(
    list(row['geometry'].exterior.coords.xy[1])),
  substation_layer.append(
    'Substation'),
  substation_id.append(
    row['ID'])
  ) for index, row in micropolis_substation.iterrows() ]

plot_substation = p_power.patches('x', 'y', source = ColumnDataSource(dict(x = x, y = y, 
                 substation_layer = substation_layer, substation_id = substation_id)), 
                           legend_field="substation_layer",
                           hatch_pattern = '>',
                           hatch_color= "lightgray",
                           fill_color= 'whitesmoke',
                           fill_alpha = 1,  
                           line_color = "darkgrey", 
                           muted_color='whitesmoke', muted_alpha=0.2,
                           line_width = 0.7)

In [33]:
# substation buses
micropolis_subbuses = micropolis_buses[micropolis_buses.layer == 'Substation buses']

x, y, subbuslayer, subbus_id = [], [], [], []

[(x.append(row['x']), 
  y.append(row['y']),
  subbuslayer.append('Substation Buses'), 
  subbus_id.append(row['FID'])) for index, row in micropolis_subbuses.iterrows()]

# x.append(5000), y.append(2950), subbuslayer.append('Substation Buses'), id.append(1000)
# x.append(5250), y.append(2950), subbuslayer.append('Substation Buses'), id.append(1001)

plot_subbus = p_power.square('x', 'y', source = ColumnDataSource(dict(x = x, y = y, subbuslayer = subbuslayer, subbus_id = subbus_id)), 
                    color='orangered', 
                    muted_color='orangered', muted_alpha=0.2,
                    size = 7, 
                    legend_field="subbuslayer")

In [34]:
# lines
x, y, line_layer, voltage, id = [], [], [], [], []
[
  (x.append(
    [micropolis_buses[micropolis_buses['FID'] == row['from']].x, 
     micropolis_buses[micropolis_buses['FID'] == row['to']].x
     ]
    ), 
  y.append(
    [micropolis_buses[micropolis_buses['FID'] == row['from']].y,
     micropolis_buses[micropolis_buses['FID'] == row['to']].y
    ]
    ),
  line_layer.append(
    'Lines'
    ),
  voltage.append(
    row['Voltage']
    ),
  id.append(
    row['id']
    )
  ) for index, row in micropolis_lines.iterrows()]

plot_power_lines = p_power.multi_line('x', 'y', source = ColumnDataSource(dict(x = x, y = y, 
      line_layer = line_layer, voltage = voltage, id = id)), color = power_color, line_width = line_width,
                                muted_color=power_color, muted_alpha=0.2,
                                legend_field="line_layer")

In [35]:
# switches
x, y, switch_layer, voltage, id = [], [], [], [], []
[
  (x.append(
    [micropolis_buses[micropolis_buses['FID'] == row['from']].x, 
     micropolis_buses[micropolis_buses['FID'] == row['to']].x
     ]
    ), 
  y.append(
    [micropolis_buses[micropolis_buses['FID'] == row['from']].y,
     micropolis_buses[micropolis_buses['FID'] == row['to']].y
    ]
    ),
  switch_layer.append(
    'Switches'
    ),
  voltage.append(
    row['Voltage']
    ),
  id.append(
    row['id']
    )
  ) for index, row in micropolis_switches.iterrows()]


plot_power_switches = p_power.multi_line('x', 'y', source = ColumnDataSource(dict(x = x, y = y, 
      switch_layer = switch_layer, voltage = voltage, id = id)), color = power_color, line_dash = 'dashed', line_width = line_width,
                                   muted_color=power_color, muted_alpha=0.2, 
                                   legend_field="switch_layer")

In [36]:
micropolis_subtransformer.head()

,id,from,to,Voltage,layer,path,sublayer,geometry
378,2002,1001,1000,None,Transformers,None,NaN,"LINESTRING (4941.554 3100.716, 4879.784 3100.7..."


In [37]:
# transformers
x, y, transfo_layer, voltage_up, voltage_down, id = [], [], [], [], [], []
[
  (x.append(
    [micropolis_buses[micropolis_buses['FID'] == row['HubName']].x, 
     int(row['Centroid'].split(',')[0])
     ]
    ), 
  y.append(
    [micropolis_buses[micropolis_buses['FID'] == row['HubName']].y,
     int(row['Centroid'].split(',')[1])
    ]
    ),
  transfo_layer.append(
    'Transformers'),
  voltage_up.append(
    '13.8 kV'
    ),
  voltage_down.append(
    row['voltage']
    ),
  id.append(
    row['ID']
    )
  ) for index, row in micropolis_buildings.iterrows()]

[
  (x.append(
    [micropolis_buses[micropolis_buses['FID'] == row['from']].x, 
     micropolis_buses[micropolis_buses['FID'] == row['to']].x
     ]
    ), 
  y.append(
    [micropolis_buses[micropolis_buses['FID'] == row['from']].y,
     micropolis_buses[micropolis_buses['FID'] == row['to']].y
    ]
    ),
  transfo_layer.append(
    'Transformers'),
  voltage_up.append(
    '138 kV'
    ),
  voltage_down.append(
    '13.8 kV'
    ),
  id.append(
    row['id']
    )
  ) for index, row in micropolis_subtransformer.iterrows()]


plot_power_transfo = p_power.multi_line('x', 'y', source = ColumnDataSource(dict(x = x, y = y, 
      voltage_down = voltage_down, voltage_up = voltage_up, transfo_layer = transfo_layer, id = id)), color = power_color, line_dash = 'dotted',
                                  muted_color=power_color, muted_alpha=0.2,  line_width = line_width,
                                  legend_field="transfo_layer")

In [38]:
#buildings
x, y, facility, building_type, population, load, connection, building_id = [], [], [], [], [], [], [], []
[
  (x.append(
    list(row['geometry'].exterior.coords.xy[0])), 
  y.append(
    list(row['geometry'].exterior.coords.xy[1])),
  facility.append(
    row['FACILITY_T']),
  building_type.append(
    row['type']),
  population.append(
    row['POP']),
  load.append(
    row['Pabc']),
  connection.append(
    row['phases']),
  building_id.append(
    row['ID'])
  ) for index, row in micropolis_buildings.iterrows() ]

building_palette = factor_cmap('type', palette = building_pal[5], factors = np.unique(np.array(building_type)))

plot_buildings = p_power.patches('x', 'y', source = ColumnDataSource(dict(x = x, y = y, 
                 facility = facility, type = building_type, population = population, 
                 load = load, connection = connection, id = building_id)), 
                           legend_field="type",
                           fill_color= building_palette,
                           fill_alpha = buildings_alpha,  
                           line_color = "white", 
                           muted_color=building_palette, muted_alpha=0.2,
                           line_width = 0.7)

In [39]:
bus_hover = HoverTool(renderers=[plot_bus])
bus_hover.tooltips = [('Bus ID', 'P_B@id'), ('Category', '@layer')]
p_power.add_tools(bus_hover)

subbus_hover = HoverTool(renderers=[plot_subbus])
subbus_hover.tooltips = [('Substation Bus ID', 'P_B@subbus_id'), ('Category', '@subbuslayer')]
p_power.add_tools(subbus_hover)

building_hover = HoverTool(renderers=[plot_buildings])
building_hover.tooltips = [('Building ID', 'P_LO@id'), ('Building type', '@facility'), ('Zone', '@type'), ('Population', '@population'),
                           ('Load', '@load kW'), ('Connection type', '@connection-phase')]
p_power.add_tools(building_hover)

power_line_hover = HoverTool(renderers=[plot_power_lines])
power_line_hover.tooltips = [('Line ID', 'P_L@id'), ('Category', '@line_layer')]
p_power.add_tools(power_line_hover)

power_switch_hover = HoverTool(renderers=[plot_power_switches])
power_switch_hover.tooltips = [('Switch ID', 'P_S@id'), ('Category', '@switch_layer')]
p_power.add_tools(power_switch_hover)

power_transfo_hover = HoverTool(renderers=[plot_power_transfo])
power_transfo_hover.tooltips = [('Transformer ID', 'P_TF@id'), ('Category', '@transfo_layer'), ('High voltage', '@voltage_up'), ('Low voltage', '@voltage_down')]
p_power.add_tools(power_transfo_hover)


In [40]:
p_power.legend.location = "top_left"
p_power.legend.click_policy="mute"
p_power.legend.background_fill_color = "gainsboro"
p_power.legend.background_fill_alpha = 0.2

# Micropolis Water Network

In [41]:
micropolis_waterjuncs = micropolis_waternodes[micropolis_waternodes.category.isin(['Intermediary node', 'Terminal node', 'Hydrant node'])]
micropolis_tanks = micropolis_waternodes[micropolis_waternodes.category.isin(['Tank'])]
micropolis_reservoirs = micropolis_waternodes[micropolis_waternodes.category.isin(['Reservoir'])]

In [42]:
# junctions
x, y, wnode_layer, wnode_category, wnode_id = [], [], [], [], []
[(x.append(row['x']), 
  y.append(row['y']),
  wnode_layer.append('Water nodes'), 
  wnode_category.append(row['category']),
  wnode_id.append(row['node'])) for index, row in micropolis_waterjuncs.iterrows()]

plot_water_juncs = p_water.square('x', 'y', source = ColumnDataSource(dict(x = x, y = y, wnode_layer = wnode_layer, wnode_category = wnode_category, id = wnode_id)), 
                    color=power_color, alpha = 0.6,
                    muted_color=power_color, muted_alpha=0.2,
                    size = 3, 
                    legend_field="wnode_layer")

In [43]:
# tanks
x, y, wtank_layer, wtank_id = [], [], [], []
[(x.append(row['x']), 
  y.append(row['y']),
  wtank_layer.append('Water tanks'), 
  wtank_id.append(row['node'])) for index, row in micropolis_tanks.iterrows()]

plot_water_tanks = p_water.square('x', 'y', source = ColumnDataSource(dict(x = x, y = y, wtank_layer = wtank_layer, id = wtank_id)), 
                    color='orangered', alpha = 1,
                    muted_color='orangered', muted_alpha=0.2,
                    size = 10, 
                    legend_field="wtank_layer")

In [44]:
# reservoirs
x, y, wres_layer, wres_id = [], [], [], []
[(x.append(row['x']), 
  y.append(row['y']),
  wres_layer.append('Reservoirs'), 
  wres_id.append(row['node'])) for index, row in micropolis_reservoirs.iterrows()]

plot_water_res = p_water.square('x', 'y', source = ColumnDataSource(dict(x = x, y = y, wres_layer = wres_layer, id = wres_id)), 
                    color='dodgerblue', alpha = 0.6,
                    muted_color='dodgerblue', muted_alpha=0.2,
                    size = 25, 
                    legend_field="wres_layer")

In [45]:
# Water pipes 
x, y, pipe_category, pipe_layer, material, diameter, roughness = [], [], [], [], [], [], []
wpipe_id = []
[
  (x.append(
    [micropolis_waternodes[micropolis_waternodes['node'] == row['from']].x, 
     micropolis_waternodes[micropolis_waternodes['node'] == row['to']].x
     ]
    ), 
  y.append(
    [micropolis_waternodes[micropolis_waternodes['node'] == row['from']].y,
     micropolis_waternodes[micropolis_waternodes['node'] == row['to']].y
    ]
    ),
  pipe_layer.append(
    'Water pipes'
    ),
  pipe_category.append(
    row['category']
    ),
  material.append(
    row['material']),
  diameter.append(
    row['diameter']),
  roughness.append(
    row['roughness']),
  wpipe_id.append(
    row['ID']
    )
  ) for index, row in micropolis_pipes.iterrows()]

water_palette = factor_cmap('pipe_category', palette = palette[0:4], factors = np.unique(np.array(pipe_category)))

plot_water_pipes = p_water.multi_line('x', 'y', source = ColumnDataSource(dict(x = x, y = y, 
      pipe_layer = pipe_layer, pipe_category = pipe_category, material = material, diameter = diameter, 
      roughness = roughness, id = wpipe_id)),  line_alpha = 1, line_width = water_width,
                                muted_color=water_palette, muted_alpha=0.2,
                                line_color = water_palette,
                                legend_field="pipe_category")

In [46]:
# Water pumps
x, y, pump_category, pump_id = [], [], [], []
[
  (x.append(
    [micropolis_waternodes[micropolis_waternodes['node'] == row['from']].x, 
     micropolis_waternodes[micropolis_waternodes['node'] == row['to']].x
     ]
    ), 
  y.append(
    [micropolis_waternodes[micropolis_waternodes['node'] == row['from']].y,
     micropolis_waternodes[micropolis_waternodes['node'] == row['to']].y
    ]
    ),
  pump_category.append(
    'Water pumps'
    ),
  pump_id.append(
    row['ID']
    )
  ) for index, row in micropolis_pumps.iterrows()]

plot_water_pumps = p_water.multi_line('x', 'y', source = ColumnDataSource(dict(x = x, y = y, 
      pump_category = pump_category, id = pump_id)),  line_alpha = 1, line_width = water_width,
                                muted_color=water_color, muted_alpha=0.2,
                                line_color = water_color,
                                line_dash = 'dashed',
                                legend_field="pump_category")

In [47]:
# #buildings
x, y, facility, building_type, population, load, connection, building_id = [], [], [], [], [], [], [], []
[
  (x.append(
    list(row['geometry'].exterior.coords.xy[0])), 
  y.append(
    list(row['geometry'].exterior.coords.xy[1])),
  facility.append(
    row['FACILITY_T']),
  building_type.append(
    row['type']),
  population.append(
    row['POP']),
  load.append(
    row['Pabc']),
  connection.append(
    row['phases']),
  building_id.append(
    row['ID'])
  ) for index, row in micropolis_buildings.iterrows() ]

building_palette = factor_cmap('building_type', palette = building_pal[5], factors = np.unique(np.array(building_type)))

plot_buildings = p_water.patches('x', 'y', source = ColumnDataSource(dict(x = x, y = y, 
                 facility = facility, building_type = building_type, population = population, 
                 load = load, connection = connection, building_id = building_id)), 
                           legend_field="building_type",
                           fill_color= building_palette,
                           fill_alpha = buildings_alpha,  
                           line_color = "white", 
                           muted_color=building_palette, muted_alpha=0.2,
                           line_width = 0.7)

In [48]:
#streams
x, y, stream_category, stream_id = [], [], [], []

[
  (x.append(
    list(row['geometry'].coords.xy[0])), 
  y.append(
    list(row['geometry'].coords.xy[1])),
  stream_category.append(
    'Streams'),
  stream_id.append(
    row['ID'])
  ) for index, row in micropolis_streams.iterrows() ]

plot_streams = p_water.multi_line('x', 'y', source = ColumnDataSource(dict(x = x, y = y, category = stream_category, stream_id = stream_id)), 
                           legend_field="category",
                           line_color = "skyblue", alpha = 0.7,
                           muted_color='skyblue', muted_alpha=0.2,
                           line_width = 3)



In [49]:
#water bodies
x, y, waterbody_category, waterbody_id = [], [], [], []
[
  (x.append(
    list(row['geometry'].exterior.xy[0])), 
  y.append(
    list(row['geometry'].exterior.xy[1])),
  waterbody_category.append(
    'Lake'),
  waterbody_id.append(
    row['ID'])
  ) for index, row in micropolis_waterbodies.iterrows() ]

plot_waterbody = p_water.patches('x', 'y', source = ColumnDataSource(dict(x = x, y = y, waterbody_category = waterbody_category, waterbody_id = waterbody_id)),
                           fill_color = "skyblue", alpha = 0.7,
                           muted_color='skyblue', muted_alpha=0.2,
                           line_width = 3)



In [50]:
building_hover = HoverTool(renderers=[plot_buildings])
building_hover.tooltips = [('Building ID', 'P_LO@building_id'), ('Building type', '@facility'), ('Zone', '@building_type'), ('Population', '@population'),
                           ('Load', '@load kW'), ('Connection type', '@connection-phase')]
p_water.add_tools(building_hover)

streams_hover = HoverTool(renderers=[plot_streams])
streams_hover.tooltips = [('Stream ID', '@stream_id'), ('Layer', '@category')]
p_water.add_tools(streams_hover)

waterbody_hover = HoverTool(renderers=[plot_waterbody])
waterbody_hover.tooltips = [('Lake ID', '@waterbody_id'), ('Layer', '@category')]
p_water.add_tools(waterbody_hover)

water_junc_hover = HoverTool(renderers=[plot_water_juncs])
water_junc_hover.tooltips = [('Node ID', '@id'), ('Layer', '@wnode_layer'), ('Category', '@wnode_category')]
p_water.add_tools(water_junc_hover)

water_res_hover = HoverTool(renderers=[plot_water_res])
water_res_hover.tooltips = [('Node ID', '@id'), ('Layer', '@wres_layer')]
p_water.add_tools(water_res_hover)

water_tank_hover = HoverTool(renderers=[plot_water_tanks])
water_tank_hover.tooltips = [('Node ID', '@id'), ('Layer', '@wtank_layer')]
p_water.add_tools(water_tank_hover)

water_pipes_hover = HoverTool(renderers=[plot_water_pipes])
water_pipes_hover.tooltips = [('Pipe ID', '@id'), ('Layer', 'pipe_layer'), ('Category', '@pipe_category'), ('Material', '@material'),('Diameter', '@diameter in.'), ('Roughness', '@roughness (DW-coef.')]
p_water.add_tools(water_pipes_hover)

water_pumps_hover = HoverTool(renderers=[plot_water_pumps])
water_pumps_hover.tooltips = [('Pump ID', '@id'), ('Layer', '@pump_category')]
p_water.add_tools(water_pumps_hover)

In [51]:
p_water.legend.location = "top_left"
p_water.legend.click_policy="mute"
p_water.legend.background_fill_color = "gainsboro"
p_water.legend.background_fill_alpha = 0.2

In [52]:
power_panel = Panel(child=p_power, title='Power')
water_panel = Panel(child=p_water, title='Water')
transpo_panel = Panel(child=p_transpo, title='Transportation')


tabs = Tabs(tabs=[power_panel, water_panel, transpo_panel])
show(tabs)

In [53]:
output_file("../../infrarisk/data/networks/micropolis/micropolis_networks.html", title="Micropolis networks")
save(tabs)

'c:\\Users\\srijith\\Dropbox\\Intra-CREATE Seed Grant\\Integrated Model\\dreaminsg_integrated_model\\infrarisk\\data\\networks\\micropolis\\micropolis_networks.html'